# Chapter 15: 모니터링 및 프로덕션

## 1. 학습 목표

* **W&B(Weights & Biases)**를 연동하여 학습 및 추론 지표를 실시간으로 대시보드에서 시각화한다.
* **vLLM**을 사용하여 고성능 추론 서버를 구축하고 OpenAI 호환 API를 제공한다.
* 데이터 드리프트(Data Drift)를 감지하고 모델을 **재학습(Re-training)**하는 전략을 수립한다.

## 2. W&B 모니터링 구축

학습 중 손실(Loss) 변화뿐만 아니라, 실제 생성된 텍스트의 품질을 추적하기 위해 W&B를 적극 활용해야 한다.

### 2.1 W&B 연동 코드

`SFTTrainer`나 `Trainer` 사용 시 `report_to="wandb"` 설정 하나로 자동 로깅이 가능하지만, 생성된 텍스트 샘플을 보기 위해서는 커스텀 콜백이 필요하다.

In [1]:
import wandb
from transformers import TrainerCallback

# 1. W&B 초기화
wandb.init(project="gpt-oss-20b-finetuning", name="production-run-v1")

# 2. 샘플 생성 로깅을 위한 콜백 정의
class TextGenerationCallback(TrainerCallback):
    def __init__(self, tokenizer, prompt, model):
        self.tokenizer = tokenizer
        self.prompt = prompt
        self.model = model

    def on_evaluate(self, args, state, control, **kwargs):
        """평가 단계마다 모델이 생성한 텍스트를 W&B 테이블에 기록한다."""
        inputs = self.tokenizer(self.prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(**inputs, max_new_tokens=100)
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # W&B Table에 기록 (Step, Prompt, Output)
        wandb.log({
            "generated_samples": wandb.Table(
                columns=["step", "prompt", "output"],
                data=[[state.global_step, self.prompt, generated_text]]
            )
        })

print("W&B 모니터링 설정 완료: 학습 지표 및 생성 텍스트 추적")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kubwai to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


W&B 모니터링 설정 완료: 학습 지표 및 생성 텍스트 추적


## 3. 고성능 서빙: vLLM

프로덕션 환경에서는 HuggingFace `pipeline`보다 **vLLM**을 사용하는 것이 표준이다. vLLM은 **PagedAttention** 알고리즘을 통해 메모리 효율을 극대화하고 처리량(Throughput)을 비약적으로 높여준다.

### 3.1 vLLM 서버 구동 (CLI)

병합된 모델(Merged Model)을 vLLM으로 로드하여 OpenAI 호환 API 서버를 띄운다.

```bash
# 1. vLLM 설치
# pip install vllm

# 2. 서버 실행 명령어
# --model: 병합된 모델 경로 또는 HF Hub ID
# --quantization: 4-bit 모델인 경우 awq 또는 bitsandbytes 설정 (필요 시)
# --gpu-memory-utilization: VRAM 점유율 설정 (기본 0.9)
python -m vllm.entrypoints.openai.api_server \
    --model ./GPT-OSS-20B-Merged \
    --host 0.0.0.0 \
    --port 8897 \
    --gpu-memory-utilization 0.95 \
    --dtype bfloat16

```

### 3.2 클라이언트 요청 테스트

서버가 띄워지면 `openai` 라이브러리를 사용해 표준적인 방식으로 요청을 보낼 수 있다.

In [11]:
from openai import OpenAI

# vLLM은 OpenAI API와 완벽하게 호환된다.
client = OpenAI(
    base_url="http://localhost:8897/v1",
    api_key="EMPTY" # vLLM 로컬 실행 시 키 불필요
)

response = client.chat.completions.create(
    model="./GPT-OSS-20B-Merged",
    messages=[
        {"role": "system", "content": "당신은 유용한 AI 비서입니다."},
        {"role": "user", "content": "LLM에서 MLOps가 왜 필요한지 설명해줘."}
    ],
    temperature=0.7,
    max_tokens=200
)

print(f"vLLM 응답: {response.choices[0].message.content}")

vLLM 응답: ## LLM(대형 언어 모델)에서 MLOps가 꼭 필요한 이유

| 구분 | 기존 ML | LLM | MLOps 필요성 |
|------|---------|-----|---------------|
| **데이터** | 소량의 라벨링된 데이터 → 작은 버전 관리 | 10억~수십억 개의 토큰 → 데이터 파이프라인, 전처리, 샘플링, 레이블링이 복잡 | **데이터


## 4. 재학습(Re-training) 전략

데이터는 시간이 지날수록 변한다(Data Drift). 모델 성능을 유지하기 위해 주기적인 재학습 파이프라인이 필수적이다.

### 4.1 재학습 시점 결정

1. **성능 기반**: 사용자 피드백(Thumbs up/down)에서 부정 평가 비율이 임계치(예: 10%)를 넘을 때.
2. **기간 기반**: 매주 또는 매월 정기적으로 최신 데이터를 반영.
3. **데이터 기반**: 새로운 전문 용어나 법률이 대거 등장했을 때.

### 4.2 망각 방지 (Continual Learning)

새로운 데이터만 학습하면 과거 지식을 잊는 '재앙적 망각'이 발생한다. 이를 막기 위해 **Replay Buffer** 기법을 사용한다.

In [5]:
from datasets import concatenate_datasets

def prepare_retraining_dataset(new_dataset, old_dataset, replay_ratio=0.2):
    """
    새로운 데이터셋에 과거 데이터 일부를 섞어 망각을 방지한다.
    """
    # 과거 데이터에서 일부만 샘플링 (Replay)
    old_samples = old_dataset.shuffle(seed=42).select(range(int(len(new_dataset) * replay_ratio)))

    # 데이터 병합
    combined_dataset = concatenate_datasets([new_dataset, old_samples])
    combined_dataset = combined_dataset.shuffle(seed=42)

    print(f"재학습 데이터 준비 완료: 신규 {len(new_dataset)} + 구버전 {len(old_samples)}")
    return combined_dataset

# 이후 SFTTrainer로 재학습 진행

Re-traing 전략 중의 하나인 Post-training은 17장에서 다룰 예정이다.

## 5. 요약

이 챕터에서는 학습된 모델을 살아있는 서비스로 만들기 위한 핵심 기술을 다루었다.

1. **W&B**: 단순 로그를 넘어 생성 결과물까지 시각화하여 품질을 관리한다.
2. **vLLM**: PagedAttention 기술로 높은 동시 접속 처리가 가능한 API 서버를 구축했다.
3. **지속적 학습**: Replay 기법을 통해 과거 지식을 유지하면서 최신 트렌드를 반영하는 파이프라인을 설계했다.

이제 마지막 단계인 **Chapter 16: MoE (Mixture of Experts) Finetuning**으로 넘어가, 최신 아키텍처인 전문가 혼합 모델을 튜닝하는 방법을 다룬다.